In [5]:
# -*- coding: utf-8 -*-
"""
red_mercado.py

Esta red neuronal predice dos eventos en un mercado simulado:
1. subida_brusca_precio  (subida abrupta de precios)
2. escasez

Se ha mejorado la legibilidad de las predicciones y se añaden métricas de desempeño.
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score
)
from keras import Sequential, Input
from keras.layers import Dense, Dropout, Normalization
from keras.metrics import Recall





In [ ]:
# 1. Carga de datos
# ----------------
datos = pd.read_csv("simulacionEconomica.csv")


In [6]:
# 2. Separar atributos y objetivo
# --------------------------------
atributos = datos.loc[:, 'demanda_preajuste':'volatilidad_precio']
objetivo = datos.loc[:, 'subida_brusca_precio':'escasez']



In [7]:
# 3. Dividir en entrenamiento y prueba
# ------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    atributos, objetivo,
    test_size=0.2,
    random_state=42
)



In [ ]:
# 4. Normalización
# ----------------
normalizador = Normalization()
normalizador.adapt(X_train.to_numpy())
np.mean(normalizador(X_train), axis=0)
np.mean(normalizador(X_train), axis=0)

In [9]:
# 5. Definición de la red
# -----------------------
red_mercado = Sequential([
    Input(shape=(5,)),         # 5 variables de entrada
    normalizador,              # normaliza cada batch
    Dense(64, activation='relu'),  # capa oculta amplia
    Dropout(0.2),                  # evita sobreajuste
    Dense(32, activation='relu'),  # capa oculta más pequeña
    Dense(2, activation='sigmoid') # 2 salidas binarias
])



In [10]:
# 6. Compilación
# --------------
red_mercado.compile(
    optimizer='SGD',
    loss='binary_crossentropy',
    metrics=['binary_accuracy', Recall()]
)



In [11]:
# 7. Entrenamiento
# ----------------
historial = red_mercado.fit(
    X_train, y_train,
    batch_size=128,
    epochs=20,
    validation_split=0.2,
    verbose=1
)



Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.5155 - loss: 0.6907 - recall_2: 0.6122 - val_binary_accuracy: 0.8156 - val_loss: 0.5777 - val_recall_2: 0.4077
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy: 0.8082 - loss: 0.5548 - recall_2: 0.3855 - val_binary_accuracy: 0.8223 - val_loss: 0.4806 - val_recall_2: 0.4216
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8312 - loss: 0.4625 - recall_2: 0.4276 - val_binary_accuracy: 0.8464 - val_loss: 0.4072 - val_recall_2: 0.5144
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8485 - loss: 0.3962 - recall_2: 0.5168 - val_binary_accuracy: 0.8689 - val_loss: 0.3566 - val_recall_2: 0.6084
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8639 - loss: 0.3515 - recall_2: 0.5829 - val_binary_accuracy: 0.8740 - val_loss: 0.3245 - val_recall_2: 0.6477
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8725 - 

In [14]:
# 8. Evaluación en conjunto de prueba
# -----------------------------------
y_pred_prob = red_mercado.predict(X_test)
# Probabilidades a etiquetas 0/1 con umbral 0.5
y_pred = (y_pred_prob >= 0.5).astype(bool)

# Métricas globales (promedio macro)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred_prob, average='macro')

print(f"Accuracy (macro): {accuracy:.4f}")
print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"ROC AUC (macro): {roc_auc:.4f}")

# Reporte por clase
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['subida_brusca', 'escasez']))

# Matrices de confusión
from sklearn.metrics import multilabel_confusion_matrix

# y_test e y_pred tienen forma (n_muestras, 2)
cms = multilabel_confusion_matrix(y_test, y_pred)

# cms[0]  → matriz para la etiqueta “subida brusca”
# cms[1]  → matriz para la etiqueta “escasez”

print("Matriz de confusión — subida brusca:")
print(cms[0])
print("\nMatriz de confusión — escasez:")
print(cms[1])




219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy (macro): 0.7671
Precision (macro): 0.8623
Recall (macro): 0.7724
ROC AUC (macro): 0.9315

Classification Report:
               precision    recall  f1-score   support

subida_brusca       0.78      0.65      0.71      2833
      escasez       0.94      0.89      0.92      1164

    micro avg       0.83      0.72      0.77      3997
    macro avg       0.86      0.77      0.81      3997
 weighted avg       0.83      0.72      0.77      3997
  samples avg       0.26      0.26      0.26      3997

Matriz de confusión — subida brusca:
[[3647  520]
 [ 990 1843]]

Matriz de confusión — escasez:
[[5775   61]
 [ 123 1041]]


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [16]:
# 9. Lectura fácil de predicciones nuevas
# ---------------------------------------
# Tomamos 10 muestras de ejemplo para ver probabilidades y clases
muestra = datos.iloc[1510:1520]
prob_muestra = red_mercado.predict(muestra.loc[:, 'demanda_preajuste':'volatilidad_precio'])
pred_muestra = (prob_muestra >= 0.5).astype(bool)

# Construimos DataFrame con los resultados
df_pred = pd.DataFrame({
    'demanda_preajuste': muestra.demanda_preajuste,
    'volatilidad_precio': muestra.volatilidad_precio,
    'prob_subida_brusca': prob_muestra[:, 0],
    'prob_escasez': prob_muestra[:, 1],
    'pred_subida_brusca': pred_muestra[:, 0],
    'pred_escasez': pred_muestra[:, 1]
})

print("\nPredicciones de ejemplo:")
print(df_pred.to_string(index=False))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Predicciones de ejemplo:
 demanda_preajuste  volatilidad_precio  prob_subida_brusca  prob_escasez  pred_subida_brusca  pred_escasez
            2427.0             1.34536            0.709754      0.117751                True         False
            2487.0             1.46544            0.391306      0.017777               False         False
            2519.0             1.53623            0.450881      0.019967               False         False
            2528.0             1.56445            0.303861      0.011350               False         False
            2617.0             1.63936            0.426541      0.017595               False         False
            2688.0             1.82962            0.710804      0.086285                True         False
            2676.0             2.00250            0.684864      0.053734                True         False
            2693.0             2.01184            0.180336      0.003504        